In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegressionCV
%matplotlib inline

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score

from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.grid_search import GridSearchCV

from pprint import pprint 

from sklearn.ensemble import BaggingClassifier
from sklearn import model_selection
from random import seed
from sklearn.model_selection import cross_val_predict

from sklearn.ensemble import RandomForestClassifier
from itertools import product
from collections import OrderedDict
from itertools import product

from sklearn import model_selection
from sklearn.svm import SVC
from sklearn import neighbors
from sklearn.ensemble import VotingClassifier

from sklearn.model_selection import train_test_split
from __future__ import print_function # For Python 2 / 3 compatability
from sklearn.metrics import confusion_matrix
import itertools

from sklearn.decomposition import PCA

from sklearn import discriminant_analysis
from sklearn.neighbors import KNeighborsClassifier as KNN

import warnings
warnings.filterwarnings("ignore")

/Users/ThomasHsueh/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/ThomasHsueh/anaconda/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


#### Load prepared dataset

In [2]:
ADNI_merge = pd.read_csv('ADNIMERGE.csv')
ADNI2 = ADNI_merge[ADNI_merge['COLPROT']=='ADNI2']
print("ADNI2 shape: ", ADNI2.shape)

ADNI2_impute = pd.read_csv('ADNI2_mb_impute_v2.csv')
print("ADNI2_impute shape: ", ADNI2_impute.shape)

ADNI2_remove = pd.read_csv('ADNI2_remove_all_missing_v2.csv')
print("ADNI2_remove shape: ", ADNI2_remove.shape)

ADNI2 shape:  (6937, 94)
ADNI2_impute shape:  (6937, 120)
ADNI2_remove shape:  (6937, 55)


In [3]:
for c in ADNI2_impute.columns:
    print(c)

AGE
FDG
AV45
CDRSB
ADAS11
ADAS13
MMSE
RAVLT_immediate
RAVLT_learning
RAVLT_forgetting
RAVLT_perc_forgetting
FAQ
MOCA
EcogPtMem
EcogPtLang
EcogPtVisspat
EcogPtPlan
EcogPtOrgan
EcogPtDivatt
EcogPtTotal
EcogSPMem
EcogSPLang
EcogSPVisspat
EcogSPPlan
EcogSPOrgan
EcogSPDivatt
EcogSPTotal
Ventricles
Hippocampus
WholeBrain
Entorhinal
Fusiform
MidTemp
ICV
CDRSB_bl
ADAS11_bl
ADAS13_bl
MMSE_bl
RAVLT_immediate_bl
RAVLT_learning_bl
RAVLT_forgetting_bl
RAVLT_perc_forgetting_bl
FAQ_bl
Ventricles_bl
Hippocampus_bl
WholeBrain_bl
Entorhinal_bl
Fusiform_bl
MidTemp_bl
ICV_bl
MOCA_bl
EcogPtMem_bl
EcogPtLang_bl
EcogPtVisspat_bl
EcogPtPlan_bl
EcogPtOrgan_bl
EcogPtDivatt_bl
EcogPtTotal_bl
EcogSPMem_bl
EcogSPLang_bl
EcogSPVisspat_bl
EcogSPPlan_bl
EcogSPOrgan_bl
EcogSPDivatt_bl
EcogSPTotal_bl
FDG_bl
AV45_bl
Years_bl
bl
m03
m06
m102
m108
m114
m12
m120
m126
m18
m24
m30
m36
m42
m48
m54
m60
m66
m72
m78
m84
m90
m96
AD
CN
EMCI
LMCI
SMC
Female
Male
Hisp/Latino
Not Hisp/Latino
Am Indian/Alaskan
Asian
Black
Hawaiian/Oth

In [4]:
ADNI2_remove.columns

Index(['RID', 'PTID', 'SITE', 'COLPROT', 'ORIGPROT', 'EXAMDATE', 'DX_bl',
       'AGE', 'EXAMDATE_bl', 'CDRSB_bl', 'MMSE_bl', 'Years_bl', 'Month_bl',
       'Month', 'M', 'update_stamp', 'bl', 'm03', 'm06', 'm102', 'm108',
       'm114', 'm12', 'm120', 'm126', 'm18', 'm24', 'm30', 'm36', 'm42', 'm48',
       'm54', 'm60', 'm66', 'm72', 'm78', 'm84', 'm90', 'm96', 'Female',
       'Male', '6', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17',
       '18', '19', '20'],
      dtype='object')

#### Narrow down to baseline-only data, and prepare X_train, y_train, X_test, and y_test

In [5]:
DROP_COL_IMPUTE = ['FDG', 'AV45', 'CDRSB', 'ADAS11', 'ADAS13', 'MMSE',
       'RAVLT_immediate', 'RAVLT_learning', 'RAVLT_forgetting',
       'RAVLT_perc_forgetting', 'FAQ', 'MOCA', 'EcogPtMem', 'EcogPtLang',
       'EcogPtVisspat', 'EcogPtPlan', 'EcogPtOrgan', 'EcogPtDivatt',
       'EcogPtTotal', 'EcogSPMem', 'EcogSPLang', 'EcogSPVisspat', 'EcogSPPlan',
       'EcogSPOrgan', 'EcogSPDivatt', 'EcogSPTotal', 'Ventricles',
       'Hippocampus', 'WholeBrain', 'Entorhinal', 'Fusiform', 'MidTemp', 'ICV',\
                  "Years_bl", "Dementia", "CN.1",\
                  'SMC', 'AD', 'EMCI', 'LMCI', 'CN']

DROP_COL_REMOVE = ['RID', 'PTID', 'SITE', 'COLPROT', 'ORIGPROT', 'EXAMDATE',\
                   'EXAMDATE_bl', 'Years_bl', 'Month_bl', 'Month', 'M', 'update_stamp']

df_impute = ADNI2_impute.drop(DROP_COL_IMPUTE, axis=1)
df_remove = ADNI2_remove.drop(DROP_COL_REMOVE, axis=1)
df_impute['PTEDUCAT'] = ADNI2["PTEDUCAT"].values
df_remove['PTEDUCAT'] = ADNI2["PTEDUCAT"].values
print(df_impute.shape)
print(df_remove.shape)
#print(df_impute.columns)
#print(df_remove.columns)

(6937, 80)
(6937, 44)


In [6]:
np.random.seed(9001)
train_percentage = 0.75
msk = np.random.rand(len(df_impute)) < train_percentage
impute_train = df_impute[msk]
impute_test  = df_impute[~msk]
remove_train = df_remove[msk]
remove_test  = df_remove[~msk]

X_train_remove = remove_train.drop("DX_bl", axis=1)
y_train_remove = remove_train["DX_bl"]
X_test_remove  = remove_test.drop("DX_bl", axis=1)
y_test_remove  = remove_test["DX_bl"]

X_train_impute = impute_train
y_train_impute = y_train_remove
X_test_impute = impute_test
y_test_impute = y_test_remove

print("For REMOVE:")
print(X_train_remove.shape)
print(y_train_remove.shape)
print(X_test_remove.shape)
print(y_test_remove.shape)

print("\nFor IMPUTE:")
print(X_train_impute.shape)
print(y_train_impute.shape)
print(X_test_impute.shape)
print(y_test_impute.shape)

For REMOVE:
(5246, 43)
(5246,)
(1691, 43)
(1691,)

For IMPUTE:
(5246, 80)
(5246,)
(1691, 80)
(1691,)


#### Running models on the remove-all-nan dataset

In [7]:
# CV Log Regression over the rest
log_ovr_remove = LogisticRegressionCV(Cs=10, cv = 5, multi_class='ovr')
log_ovr_remove.fit(X_train_remove, y_train_remove)

# CV Log Regression multinomila
log_multi_remove = LogisticRegressionCV(Cs=10, cv = 5, multi_class = 'multinomial')
log_multi_remove.fit(X_train_remove, y_train_remove)

#Linear discriminant analysis
LDA_remove = discriminant_analysis.LinearDiscriminantAnalysis()
LDA_remove.fit(X_train_remove, y_train_remove)

#Quadratic discriminant analysis
QDA_remove = discriminant_analysis.QuadraticDiscriminantAnalysis()
QDA_remove.fit(X_train_remove, y_train_remove)

# CV k-nearest neighbour
knn = KNN()
param=np.arange(1,20)
param_grid = dict(n_neighbors=param)
gs = GridSearchCV(KNN(), param_grid, cv=5, n_jobs=2)
gs.fit(X_train_remove, y_train_remove)
knn_remove = gs.best_estimator_

#CV tree classifier
n_estimators =np.arange(1, 20)
param_grid = dict(max_depth=n_estimators)
gs = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=5, n_jobs=2)
gs.fit(X_train_remove, y_train_remove)
tree_remove = gs.best_estimator_

#CV Random forest
max_features = np.arange(1,len(X_train_remove.columns))              #number of predictors at ech split
max_depth = np.arange(1,20)                                   #max depth
rf = RandomForestClassifier(n_estimators=50, max_depth=None)
param_grid = dict(max_features=max_features)
gs = GridSearchCV(rf, param_grid, cv=5, n_jobs=2)
gs.fit(X_train_remove, y_train_remove)
rf_remove = gs.best_estimator_

#Adaboost
max_depth = np.arange(1,5)
numb_estimators = [10*x for x in range(1,10)]

param_grid = {'base_estimator__max_depth':max_depth,
              'n_estimators':numb_estimators}
DTC = DecisionTreeClassifier()
ada = AdaBoostClassifier(base_estimator = DTC, learning_rate=.05)
gs = GridSearchCV(ada, param_grid, cv=5, n_jobs=2)
gs.fit(X_train_remove, y_train_remove)
ada_remove = gs.best_estimator_

In [8]:
models_remove = [log_ovr_remove, log_multi_remove, LDA_remove, QDA_remove, knn_remove,\
          tree_remove, rf_remove, ada_remove]
models_name = ['logistic OVR', 'logistic Multi', 'LDA', 'QDA', 'KNN', 'tree', 'Random forest', 'AdaBoost']

test_score_remove =[]
zipped = zip(models_remove, models_name)
print("For models running on remove-all-nan dataset:")
print("=============================================")
for i,r in zipped:
    print(r+' train accurcy is %0.3f' % (i.score (X_train_remove,y_train_remove)))
    print(r+' test accurcy is %0.3f' % (i.score (X_test_remove,y_test_remove)))
    print("\n")
    test_score_remove.append(i.score (X_test_remove,y_test_remove))

For models running on remove-all-nan dataset:
logistic OVR train accurcy is 0.760
logistic OVR test accurcy is 0.742


logistic Multi train accurcy is 0.772
logistic Multi test accurcy is 0.760


LDA train accurcy is 0.716
LDA test accurcy is 0.692


QDA train accurcy is 0.272
QDA test accurcy is 0.287


KNN train accurcy is 0.761
KNN test accurcy is 0.705


tree train accurcy is 0.734
tree test accurcy is 0.708


Random forest train accurcy is 1.000
Random forest test accurcy is 0.860


AdaBoost train accurcy is 0.750
AdaBoost test accurcy is 0.723




#### Running models on the imputed dataset

In [9]:
# CV Log Regression over the rest
log_ovr_impute = LogisticRegressionCV(Cs=10, cv = 5, multi_class='ovr')
log_ovr_impute.fit(X_train_impute, y_train_impute)

# CV Log Regression multinomila
log_multi_impute = LogisticRegressionCV(Cs=10, cv = 5, multi_class = 'multinomial')
log_multi_impute.fit(X_train_impute, y_train_impute)

#Linear discriminant analysis
LDA_impute = discriminant_analysis.LinearDiscriminantAnalysis()
LDA_impute.fit(X_train_impute, y_train_impute)

#Quadratic discriminant analysis
QDA_impute = discriminant_analysis.QuadraticDiscriminantAnalysis()
QDA_impute.fit(X_train_impute, y_train_impute)

# CV k-nearest neighbour
knn = KNN()
param=np.arange(1,20)
param_grid = dict(n_neighbors=param)
gs = GridSearchCV(KNN(), param_grid, cv=5, n_jobs=2)
gs.fit(X_train_impute, y_train_impute)
knn_impute = gs.best_estimator_

#CV tree classifier
n_estimators =np.arange(1, 20)
param_grid = dict(max_depth=n_estimators)
gs = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=5, n_jobs=2)
gs.fit(X_train_impute, y_train_impute)
tree_impute = gs.best_estimator_

#CV Random forest
max_features = np.arange(1,len(X_train_impute.columns))              #number of predictors at ech split
max_depth = np.arange(1,20)                                   #max depth
rf = RandomForestClassifier(n_estimators=50, max_depth=None)
param_grid = dict(max_features=max_features)
gs = GridSearchCV(rf, param_grid, cv=5, n_jobs=2)
gs.fit(X_train_impute, y_train_impute)
rf_impute = gs.best_estimator_

#Adaboost
max_depth = np.arange(1,5)
numb_estimators = [10*x for x in range(1,10)]

param_grid = {'base_estimator__max_depth':max_depth,
              'n_estimators':numb_estimators}
DTC = DecisionTreeClassifier()
ada = AdaBoostClassifier(base_estimator = DTC, learning_rate=.05)
gs = GridSearchCV(ada, param_grid, cv=5, n_jobs=2)
gs.fit(X_train_impute, y_train_impute)
ada_impute = gs.best_estimator_

In [10]:
models_impute = [log_ovr_impute, log_multi_impute, LDA_impute, QDA_impute, knn_impute,\
          tree_impute, rf_impute, ada_impute]
models_name = ['logistic OVR', 'logistic Multi', 'LDA', 'QDA', 'KNN', 'tree', 'Random forest', 'AdaBoost']

test_score_impute =[]
zipped = zip(models_impute, models_name)
print("For models running on imputed dataset:")
print("=============================================")
for i,r in zipped:
    print(r+' train accurcy is %0.3f' % (i.score (X_train_impute,y_train_impute)))
    print(r+' test accurcy is %0.3f' % (i.score (X_test_impute,y_test_impute)))
    print("\n")
    test_score_impute.append(i.score (X_test_impute,y_test_impute))

For models running on imputed dataset:
logistic OVR train accurcy is 0.463
logistic OVR test accurcy is 0.481


logistic Multi train accurcy is 0.441
logistic Multi test accurcy is 0.455


LDA train accurcy is 0.802
LDA test accurcy is 0.792


QDA train accurcy is 0.414
QDA test accurcy is 0.425


KNN train accurcy is 0.606
KNN test accurcy is 0.567


tree train accurcy is 0.789
tree test accurcy is 0.794


Random forest train accurcy is 1.000
Random forest test accurcy is 0.989


AdaBoost train accurcy is 0.777
AdaBoost test accurcy is 0.775


